In [1]:
import numpy as np
import pandas as pd

def custom_groupby(
        df,
        groupby_cols=['search_str'],
        groupby_expressions=[
            {'new_col': '搜索次数', 'original_col': 'user_id', 'function': 'numpy.size'},
            {'new_col': '搜索人数', 'original_col': 'user_id', 'function': 'pd.Series.nunique'},
            {'new_col': '搜索平台数', 'original_col': 'platform_id', 'function': 'pd.Series.nunique'}
        ],
        improt_numpy_and_pandas=False

):
    if improt_numpy_and_pandas:
        import numpy as np
        import pandas as pd
    raw_code = [f'''df.groupby({groupby_cols},sort=False).agg(''']
    for item in groupby_expressions:
        raw_code.append(f'''{item['new_col']}=('{item['original_col']}',{item['function']}),''')
    raw_code.append(')')
    return eval(''.join(raw_code))


def custom_groupby_inner(df, col_name, percentile=80):
    df['占比'] = np.round(df[col_name] / df[col_name].sum() * 100, 2)
    df.sort_values(by='占比', ascending=False, inplace=True)
    df['cumsum'] = df['占比'].cumsum()
    if len(df[df['cumsum'] <= percentile]) > 0:
        df = df[df['cumsum'] <= percentile]
    else:
        df = df[:1]
    percentage_str = [(df.at[index, '占比'], index) for index in df.index]
    del df
    percentage_str = '、'.join(str(item) for item in percentage_str)
    return percentage_str

In [ ]:
tobe_grouped_df = sku_click_detail_ufen_v2_df.groupby('SKU_code')
    del sku_click_detail_ufen_v2_df
    sku_click = pd.DataFrame()
    for group_name, df in tobe_grouped_df:
        df_grouped = custom_groupby(df=df, groupby_cols=['SKU_code'], groupby_expressions=[
            {'new_col': '点击次数', 'original_col': 'user_id', 'function': 'np.size'},
            {'new_col': '点击人数', 'original_col': 'user_id', 'function': 'pd.Series.nunique'},
            {'new_col': '点击平台数', 'original_col': 'platform_id', 'function': 'pd.Series.nunique'}
        ])
        df_grouped_inner = custom_groupby(df=df, groupby_cols=['点击来源'], groupby_expressions=[
            {'new_col': '点击次数', 'original_col': 'user_id', 'function': 'np.size'}
        ])
        percentage = custom_groupby_inner(df=df_grouped_inner, col_name='点击次数', percentile=100)
        del df_grouped_inner
        df_grouped['来源点击次数占比（占比，来源）'] = percentage
        sku_click = pd.concat([sku_click, df_grouped])
    sku_click.reset_index(inplace=True)
    del tobe_grouped_df